In [1]:
!pip install keras-tuner

Processing c:\users\sonal\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94\future-0.18.2-py3-none-any.whl
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-py3-none-any.whl size=78941 sha256=ca32795f495c2bca121507cb4a50f3687e37a183fab2389e597144a10bceb0ac
  Stored in directory: c:\users\sonal\appdata\local\pip\cache\wheels\f9\42\e3\73f763092b16b23350dbc5b7d9b6220bdbff2944ffcc2c612b
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15359 sha256=bd2101b53dce793da727a345267f99484930a7096f603a9c3db0b4dfa92f42d6
  Stored in directory: c:\users\sonal\appdata\local\pip\cache\wheels\86\1b\58\c23af2fe683acd8edc15d5a1268f0242be1ff2cf827fe34737
Successfully built keras-tuner terminaltables


In [8]:
import pandas as pd
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)
## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
X_train.shape

(8000, 11)

In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [11]:

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='ANN',
    project_name='Churn1')

In [12]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 14s]
val_accuracy: 0.8629999955495199

Best val_accuracy So Far: 0.8658333420753479
Total elapsed time: 00h 02m 09s
INFO:tensorflow:Oracle triggered exit


In [15]:
models=tuner.get_best_models(num_models=5)

In [17]:
tuner.results_summary()

Results summary
Results in ANN\Churn1
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 11
units_0: 320
units_1: 256
learning_rate: 0.001
units_2: 416
units_3: 32
units_4: 480
units_5: 352
units_6: 64
units_7: 416
units_8: 320
units_9: 160
units_10: 128
units_11: 384
units_12: 160
units_13: 352
units_14: 448
units_15: 512
units_16: 352
units_17: 192
Score: 0.8658333420753479
Trial summary
Hyperparameters:
num_layers: 18
units_0: 416
units_1: 384
learning_rate: 0.0001
units_2: 288
units_3: 96
units_4: 160
units_5: 416
units_6: 64
units_7: 64
units_8: 224
units_9: 352
units_10: 320
units_11: 192
units_12: 128
units_13: 448
units_14: 128
units_15: 32
units_16: 32
units_17: 32
Score: 0.8650000095367432
Trial summary
Hyperparameters:
num_layers: 2
units_0: 416
units_1: 256
learning_rate: 0.01
units_2: 384
units_3: 480
units_4: 128
units_5: 192
units_6: 192
units_7: 288
units_8: 224
units_9: 256
units_10: 128
units_11: 480
units